In [1]:
import pandas as pd
import numpy as np
from datascience import *
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [247]:
ball_by_ball = Table().read_table('Ball_by_Ball.csv')
match_data = Table().read_table('Match.csv')
player_data = Table().read_table('Player.csv')

In [248]:
ball_by_ball.show(3)

Match_Id,Innings_Id,Over_Id,Ball_Id,Team_Batting_Id,Team_Bowling_Id,Striker_Id,Striker_Batting_Position,Non_Striker_Id,Bowler_Id,Batsman_Scored,Extra_Type,Extra_Runs,Player_dissimal_Id,Dissimal_Type,Fielder_Id
335987,1,1,1,1,2,1,1,2,14,0,legbyes,1,,,
335987,1,1,2,1,2,2,2,1,14,0,,,,,
335987,1,1,3,1,2,2,2,1,14,0,wides,1,,,


In [249]:
powerplay_overs = ball_by_ball.where('Over_Id', are.between_or_equal_to(1,6))
powerplay_overs.show(40)

Match_Id,Innings_Id,Over_Id,Ball_Id,Team_Batting_Id,Team_Bowling_Id,Striker_Id,Striker_Batting_Position,Non_Striker_Id,Bowler_Id,Batsman_Scored,Extra_Type,Extra_Runs,Player_dissimal_Id,Dissimal_Type,Fielder_Id
335987,1,1,1,1,2,1,1,2,14,0,legbyes,1,,,
335987,1,1,2,1,2,2,2,1,14,0,,,,,
335987,1,1,3,1,2,2,2,1,14,0,wides,1,,,
335987,1,1,4,1,2,2,2,1,14,0,,,,,
335987,1,1,5,1,2,2,2,1,14,0,,,,,
335987,1,1,6,1,2,2,2,1,14,0,,,,,
335987,1,1,7,1,2,2,2,1,14,0,legbyes,1,,,
335987,1,2,1,1,2,2,2,1,15,0,,,,,
335987,1,2,2,1,2,2,2,1,15,4,,,,,
335987,1,2,3,1,2,2,2,1,15,4,,,,,


In [250]:
empty_table = powerplay_overs.where('Over_Id', are.equal_to(25))
empty_table

Match_Id,Innings_Id,Over_Id,Ball_Id,Team_Batting_Id,Team_Bowling_Id,Striker_Id,Striker_Batting_Position,Non_Striker_Id,Bowler_Id,Batsman_Scored,Extra_Type,Extra_Runs,Player_dissimal_Id,Dissimal_Type,Fielder_Id


In [251]:
powerplay_overs.select(3,6,'Bowler_Id', 'Batsman_Scored', 'Extra_Runs')

Ball_Id,Striker_Id,Bowler_Id,Batsman_Scored,Extra_Runs
1,1,14,0,1
2,2,14,0,
3,2,14,0,1
4,2,14,0,
5,2,14,0,
6,2,14,0,
7,2,14,0,1
1,2,15,0,
2,2,15,4,
3,2,15,4,


In [252]:
match_id_12 = match_data.where(1, are.containing('-12')).column(0)
match_id_12

array([548311, 548312, 548313, 548314, 548315, 548316, 548317, 548318,
       548319, 548320, 548321, 548322, 548323, 548324, 548325, 548326,
       548327, 548328, 548329, 548330, 548331, 548332, 548333, 548334,
       548335, 548336, 548337, 548338, 548339, 548340, 548341, 548342,
       548344, 548346, 548347, 548348, 548349, 548350, 548351, 548352,
       548353, 548354, 548355, 548356, 548357, 548358, 548359, 548360,
       548361, 548362, 548363, 548364, 548365, 548366, 548367, 548368,
       548369, 548370, 548371, 548372, 548373, 548374, 548375, 548376,
       548377, 548378, 548379, 548380, 548381, 548382, 548383, 548384,
       548385, 548386])

In [253]:
for matchid in match_id_12:
    tbl = powerplay_overs.where(0, are.equal_to(matchid))
    empty_table.append(tbl)
    
    
    

In [254]:
empty_table.show(400)

Match_Id,Innings_Id,Over_Id,Ball_Id,Team_Batting_Id,Team_Bowling_Id,Striker_Id,Striker_Batting_Position,Non_Striker_Id,Bowler_Id,Batsman_Scored,Extra_Type,Extra_Runs,Player_dissimal_Id,Dissimal_Type,Fielder_Id
548311,1,1,1,3,7,303,1,185,194,0,wides,1,,,
548311,1,1,2,3,7,303,1,185,194,0,,,,,
548311,1,1,3,3,7,303,1,185,194,3,,,,,
548311,1,1,4,3,7,185,2,303,194,0,,,,,
548311,1,1,5,3,7,185,2,303,194,0,,,303,run out,208
548311,1,1,6,3,7,21,3,185,194,0,,,,,
548311,1,1,7,3,7,21,3,185,194,1,,,,,
548311,1,2,1,3,7,21,3,185,246,1,,,,,
548311,1,2,2,3,7,185,2,21,246,0,wides,1,,,
548311,1,2,3,3,7,185,2,21,246,0,,,,,


In [261]:
conc_table = empty_table.select('Striker_Id', 'Batsman_Scored').where(1, are.not_equal_to('Do_nothing'))
conc_table

Striker_Id,Batsman_Scored
303,0
303,0
303,3
185,0
185,0
21,0
21,1
21,1
185,0
185,0


In [265]:
arr_scores = make_array()
for i in conc_table.column(1):
    a = float(i)
    arr_scores = np.append(arr_scores, int(a))    

In [273]:
conc_table = conc_table.select(0).with_column('Batsman_scored', arr_scores)
conc_table

Striker_Id,Batsman_scored
303,0
303,0
303,3
185,0
185,0
21,0
21,1
21,1
185,0
185,0


In [274]:
score_group_tbl = conc_table.group(0, sum).relabel(1, 'Total Score')
score_group_tbl

Striker_Id,Total Score
1,127
2,134
4,16
6,323
8,99
9,96
10,40
17,109
19,145
20,4


In [278]:
out_table1 = empty_table.select('Striker_Id', 'Player_dissimal_Id').where(1, are.not_equal_to(' '))
out_table = out_table1.select('Striker_Id').with_column('Player_dismissal', [int(i) for i in out_table1.column(1)])

In [279]:
grouped_table = out_table.group(1).relabel(0, 'Striker_Id')
grouped_table

Striker_Id,count
2,1
9,1
31,1
87,1
185,1
303,1


In [280]:
joint = score_group_tbl.join('Striker_Id', grouped_table, 'Striker_Id').relabel(2, 'Dismissal Count')
final_table = joint.with_column('PP Batting Average', joint.column(1) / joint.column(2))

In [281]:
player_data.select(0,1)

Player_Id,Player_Name
1,SC Ganguly
2,BB McCullum
3,RT Ponting
4,DJ Hussey
5,Mohammad Hafeez
6,R Dravid
7,W Jaffer
8,V Kohli
9,JH Kallis
10,CL White


In [294]:
PP_B_Avg12 = final_table.join('Striker_Id', player_data.select(0,1), 'Player_Id').drop(0).select(3, 2)
PP_B_Avg12

Player_Name,PP Batting Average
BB McCullum,134
JH Kallis,96
YK Pathan,1
MK Tiwary,23
M Vijay,196
F du Plessis,232


In [316]:
def PP_B_Avg_finder(year):
    empty_table = powerplay_overs.where('Over_Id', are.equal_to(25))
    match_id = match_data.where(1, are.containing("-" + str(year)[2:])).column(0)
    for m_id in match_id:
        tbl = powerplay_overs.where(0, are.equal_to(m_id))
        empty_table.append(tbl)
    con_table = empty_table.select('Striker_Id', 'Batsman_Scored').where(1, are.not_equal_to('Do_nothing'))
    arr_scores = make_array()
    for i in con_table.column(1):
        a = float(i)
        arr_scores = np.append(arr_scores, int(a)) 
    con_table = con_table.select(0).with_column('Batsman_scored', arr_scores)
    score_group_tbl1 = con_table.group(0, sum).relabel(1, 'Total Score')
    out_table2 = empty_table.select('Striker_Id', 'Player_dissimal_Id').where(1, are.not_equal_to(' '))
    out_table3 = out_table2.select('Striker_Id').with_column('Player_dismissal', [int(i) for i in out_table2.column(1)])
    grouped_table = out_table3.group(1).relabel(0, 'Striker_Id')
    joint1 = score_group_tbl1.join('Striker_Id', grouped_table, 'Striker_Id')
    final_table = joint1.with_column('PP Batting Average', joint1.column(1) / joint1.column(2))
    PP_B_Avg = final_table.join('Striker_Id', player_data.select(0,1), 'Player_Id').drop(0).select(3, 2)
    return PP_B_Avg
    
    
        
        

In [ ]:
PP_B_Avg_12 = PP_B_Avg_finder(2012)
PP_B_Avg_13 = PP_B_Avg_finder(2013)
PP_B_Avg_14 = PP_B_Avg_finder(2014)
PP_B_Avg_15 = PP_B_Avg_finder(2015)
PP_B_Avg_16 = PP_B_Avg_finder(2016)
PP_B_Avg_17 = PP_B_Avg_finder(2017)
PP_B_Avg_18 = PP_B_Avg_finder(2018)
PP_B_Avg_19 = PP_B_Avg_finder(2019)

In [327]:
PP_B_Avg_finder(2019)

AttributeError: 'NoneType' object has no attribute 'with_column'